# Import Dependencies

In [1]:
import pandas as pd

# Preprocessing

In [60]:
# import raw data
data = pd.read_csv('County_Health_Rankings.csv')
# remove unwanted columns
df = data.drop(['Confidence Interval Lower Bound', 'Confidence Interval Upper Bound', 'Data Release Year'], axis=1)
# remove rows where State is PR or NaN
df = df.loc[df['State'] != "PR"].dropna(subset=['State'])

# change datatype of columns that should be integers
int_list = ['State code', 'County code', 'Measure id']
for col in int_list:
    df[col] = df[col].astype('int')
    
## handle rows where FIPS Code is missing
# create reference table of counties and corresponding FIPS codes
fips_df = df.dropna(subset=['fipscode'])
# keep only first row for each county name
fips_df = fips_df.drop_duplicates(subset=['County', 'County code', 'State'], keep='first')
# remove unnecessary columns
fips_df = fips_df[['State','County', 'fipscode']].copy()

# transform fips code to integer and standardize format
fips_df['fipscode'] = fips_df['fipscode'].astype('int').apply(lambda x: '{0:0>5}'.format(x))
# rename FIPS column
fips_df = fips_df.rename(columns={"fipscode": "FIPS Code"}).reset_index(drop=True)

# rejoin back onto dataframe to fill missing values
df2 = df.merge(fips_df, how="left",on=["County", "State"])

# get rid of rows where Year span = "."
df2 = df2.loc[df2['Year span'] != "."].copy()

# view final dataframe
df2

,State,County,State code,County code,Year span,Measure name,Measure id,Numerator,Denominator,Raw value,fipscode,FIPS Code
0,US,United States,0,0,2003-2005,Violent crime rate,43,1328750.667,274877117.0,483.398066,0.0,00000
1,US,United States,0,0,2004-2006,Violent crime rate,43,1340928.667,277612778.5,483.021233,0.0,00000
2,US,United States,0,0,2005-2007,Violent crime rate,43,1355853.167,280407694.7,483.529230,0.0,00000
3,US,United States,0,0,2006-2008,Violent crime rate,43,1366928.333,287614567.7,475.263942,0.0,00000
4,US,United States,0,0,2007-2009,Violent crime rate,43,1339439.333,292576281.2,457.808585,0.0,00000
...,...,...,...,...,...,...,...,...,...,...,...,...
303478,WY,Weston County,56,45,2007,Daily fine particulate matter,125,NaN,NaN,9.370000,56045.0,56045
303479,WY,Weston County,56,45,2008,Daily fine particulate matter,125,NaN,NaN,7.450000,56045.0,56045
303480,WY,Weston County,56,45,2009,Daily fine particulate matter,125,NaN,NaN,12.770000,56045.0,56045
303481,WY,Weston County,56,45,2010,Daily fine particulate matter,125,NaN,NaN,12.110000,56045.0,56045
